In [2]:
import networkx as nx
import random
from collections import Counter
import numpy as np

In [3]:
def random_surfer(path="./data/p2p-Gnutella08-mod.txt", steps=1000000, m=0.15, top=10):
    """
    Imitates a "random surfer" over nodes and lists "top" visited pages.
    path - path to adjlist. default is gnutella dataset.
    steps - how many steps. default 1_000_000
    m - dumping factor. default 0.15
    top - how . default 5
    """
    fh=open(path, 'rb')
    G=nx.read_adjlist(fh, create_using=nx.DiGraph())
    fh.close()
    # convert G.nodes to type list, which will be used later for random.choice()
    nodes = list(G.nodes)
    # create dict with nodes as keys, all values set to zero.
    occurrences = dict.fromkeys(nodes, 0)
    # start from random node
    currentNode = goToOtherRandomNode(nodes)
    # iterate for set steps
    for _ in range(steps):
        # if random number is bigger than m, then:
        if (random.random()>m):
            successors = list(G.successors(currentNode))
            # if it's dangling node, go to a random node.
            if(len(successors)==0):
                currentNode = goToOtherRandomNode(nodes)
            # if it's not, go to random successor, including self
            else:
                 currentNode = goToOtherRandomNode(successors)
        else: # go to random OTHER node. (it's stated in the task, though I think in pageRank we would not exclude current node)
            # currentNode = goToOtherRandomNode(nodes, currentNode) <- use if want to go to "a random OTHER node"
            currentNode = goToOtherRandomNode(nodes)
        # updates occurrences dict
        occurrences[currentNode]+=1
    # use Counter to obtain top n nodes (n most common)
    visitedMostOften = [node for node, _ in Counter(occurrences).most_common(top)]
    return visitedMostOften

def goToOtherRandomNode(nodes, currentNode=-1):
        newCurrentNode = random.choice(nodes)
        # use if want to go to "a random OTHER node"
        # while(newCurrentNode==currentNode):
        #     newCurrentNode = random.choice(nodes)
        return newCurrentNode


In [4]:
random_surfer(top=10)

['367', '249', '145', '264', '266', '127', '123', '5', '427', '122']

In [16]:
def pageRank(path="./data/p2p-Gnutella08-mod.txt", steps=10, m=0.15, top=False):
    """
    Warning! Function assummes that nodes are represented as integers.

    Parameters:
    path - path to the txt file with adjlist. p2p-Gnuttella08-mod.txt as default
    steps - steps of power method. 10 as default
    m - damping factor. 0.15 as default
    top - how many top to show. 10 as default

    Returns:
    Top zip of top importance scores.

    """
    #load file
    fh=open(path, 'rb')
    G=nx.read_adjlist(fh, create_using=nx.DiGraph(), nodetype=int)
    fh.close()


    #obtain number of nodes
    number_of_nodes = G.number_of_nodes()
    #obtain 1/number_of_nodes which will be used many times.
    inv_number_of_nodes = 1/number_of_nodes
    # m complement will be used to many other calculations as well
    m_compl = 1-m
    #create a dict of a structure "node: outer_degree". It's faster in finding elements.
    out_degrees = {node: G.out_degree(node) for node in G.nodes}
    #create set of dangling nodes. It is faster in finding elements.
    dangling_nodes = {node for node, out_degree in out_degrees.items() if out_degree==0}

    # start off with x_0 with the same importance score to every site
    x_k = (inv_number_of_nodes)*np.ones(number_of_nodes)

    

    for _ in range(steps):
        # From theory we know that: x_k_1 = (1-m)Ax_k + (1-m)Dx_k + mSx_k
        # Which we will compute in an order: x_k_1 = (1-m)(Dx_k+Ax_k)+mSx_k

        # computing Dx_k
        # since each row of D is the same, then every element of Dx_k equals to 1/n*sum(sum of importance of dangling nodes). we create a whole vector now.
        x_k_1 = (inv_number_of_nodes)*np.full((number_of_nodes), sum(x_k[dangling_node] for dangling_node in dangling_nodes))
        # Currently M = Dx_k

        #computing Ax_k
        # Each of the page has only one vote to vote for all pages. Therefore the vote is partitioned evenly amongst all pages, the page is voting for.
        # Which is: (1/number_of_voted_pages)
        # Vote is multiplied by their corresponding importance score (x_k[backlink])
        # Consequently, For each page, we therefore sum all of theirs backlinks "scaled votes", so we know how other pages consider this page to be important.
        # TODO: for sure it can be vectorized and then optimized with "apply_along_lines" numpy's method
        for node in G.nodes:
            j = sum((1/out_degrees[backlink])*x_k[backlink] for backlink in G.predecessors(node))
            x_k_1[node]+=j
        # M = Ax_k+Dx_k

        x_k_1 *= (m_compl)
        # M = (1-m)(Ax_k+Dx_k)
        
        # mSx_k equals always to one column vector with all 1/n, so we can assign it right know to x_k_1
        x_k_1 += m*(inv_number_of_nodes)*np.ones(number_of_nodes)
        # M = (1-m)(Ax_k+Dx_k)+mSx_k

        # We do a copy (reference would destroy it!)
        x_k = x_k_1.copy()

    # now we have vector x_k, where values at given indices correspond to their vectors.
    importance_scores = {node: x_k[node] for node in G.nodes}
    if(top):
        # return importance_scores and indices (which are names of nodes here) corresponding to top values
        highest_ranking_nodes = sorted(importance_scores.keys(), key=importance_scores.get, reverse=True)[:top]
        highest_values = [importance_scores[highest_ranking_node] for highest_ranking_node in highest_ranking_nodes]
        highest_ranking_nodes_with_values = list(zip(highest_ranking_nodes, highest_values))
        return (importance_scores, highest_ranking_nodes_with_values, highest_ranking_nodes)
    return importance_scores

In [18]:
def approx_number_of_steps_to_stabilise():
    current_approx = []
    i=1
    while(True):
        new_approx = pageRank(steps = i, top=10)[2]
        if(current_approx==new_approx):
            return i-1
        current_approx=new_approx
        i+=1
        
ideal_steps = approx_number_of_steps_to_stabilise()
ideal_steps

4

In [ ]:
%%timeit
importance_scores = pageRank(steps=ideal_steps, top=10)


In [10]:
# Get top values
importance_scores, top, _ = pageRank(steps=4, top=10)
print(sum(importance_scores.values()))
print(top)
importance_scores


1.000000000000031
[(367, 0.002354333106588876), (249, 0.002161080620257098), (145, 0.002026083712777378), (264, 0.0019705897890470374), (266, 0.0019427388809176306), (123, 0.0018509176863445635), (127, 0.0018489936951539621), (122, 0.0018334167436472783), (1317, 0.0018178105376691369), (5, 0.0018072930841105386)]


{0: 0.00010069326492726427,
 1: 0.0001092751374932969,
 2: 0.00017154656892479206,
 3: 0.0014106563096252837,
 4: 0.001536997555728457,
 5: 0.0018072930841105386,
 6: 0.0001092751374932969,
 7: 0.0014825869566469647,
 8: 0.001259589153609421,
 9: 0.0011904797933642531,
 10: 0.00030855335994983376,
 703: 0.00026459957007570297,
 826: 0.00025349634646865615,
 1097: 0.00030435537521364967,
 1287: 0.00027991703091895476,
 1591: 0.0003916450043421572,
 1895: 0.00021867704198396673,
 1896: 0.0002915487559168416,
 1897: 0.00021867704198396673,
 1898: 0.00023895744545750642,
 1899: 0.00023110291129676517,
 144: 0.0013068198392871822,
 258: 0.0002510184701346168,
 491: 0.0003586125070182115,
 1021: 0.0003680685647399968,
 1418: 0.00024144381889429142,
 1669: 0.0002852556552727484,
 1900: 0.00033015013501956706,
 1901: 0.0002866754033075808,
 1902: 0.00022948412232224593,
 1903: 0.00047718989657196103,
 121: 0.0011986584032044683,
 127: 0.0018489936951539621,
 128: 0.0012948388467643483,
 179: 0

In [4]:
fh=open("./data/p2p-Gnutella08-mod.txt", 'rb')
G=nx.read_adjlist(fh, create_using=nx.DiGraph())
fh.close()
o = nx.pagerank(G)

sorted(o, key=o.get, reverse=True)[:10]

['367', '249', '145', '264', '266', '123', '127', '122', '1317', '5']